In [1]:
import json

with open("train.json",'r',encoding='utf-8') as f:
    data = f.read()

data = json.loads(data)
print("样本数量:", len(data))

样本数量: 6315


总的样本数量为6315，

研究每个样本中问题数量：

问题数量为1的样本个数为：509

问题数量为2的样本个数为：2656

问题数量为3的样本个数为：3004

问题数量为4的样本个数为：138

问题数量为5的样本个数为：8

In [2]:
from collections import Counter

numbers = []
for d in data:
    numbers.append(len(d['Questions']))

print("问题数量分布:", Counter(numbers))


问题数量分布: Counter({3: 3004, 2: 2656, 1: 509, 4: 138, 5: 8})


查找样本中答案的分布情况

In [5]:
choice_number = []
answers_dis = []
for d in data:
    for q in d['Questions']:
        answers_dis.append(q['Answer'])
        choice_number.append(len(q['Choices']))
            
print("答案分布:", Counter(answers_dis))
print("选项个数分布:",Counter(choice_number))

答案分布: Counter({'C': 4633, 'B': 4097, 'D': 3994, 'A': 2701})
选项个数分布: Counter({4: 15329, 3: 71, 2: 25})


In [7]:
"""查看每个选项个数小于3的"""
two_choice = []
for d in data:
    for q in d['Questions']:
        if len(q['Choices'])==2:
            two_choice.append(q)
for q in two_choice[:5]:
    print(q)

{'Question': '本文的开头有两种不同的译文，你觉得哪种译文更能体现童话语言的特点？', 'Choices': ['A. 一个国王有一个独生子，国王爱他就像爱自己眼睛一样。可这个王子总是不快乐，整天整天地站在阳台上，看着远处。', 'B. 国王有一个独生子，国王很疼爱他，视若珍宝。可这个王子总是郁郁寡欢，整天站在阳台上，看着远处。'], 'Answer': 'A', 'Q_id': '007001'}
{'Question': '判断晏子朝，乘弊车，驾驽马中的朝解释为上朝是否正确', 'Choices': ['A. 正确', 'B. 错误'], 'Answer': 'A', 'Q_id': '024602'}
{'Question': '若在选文第④段括号内填写一个事实论据，下面的两个例子哪一个更好？说说你的理由。', 'Choices': ['A. 音乐家冼星海在延安时，没有钢琴供他练奏，可对音乐满怀热忱的他，以乐观的心态，用碗、碟、盆、罐等替代钢琴，经过长期酝酿，终于创作出了史诗般的作品——《黄河人合唱》。', 'B. 周恩来总理从少年时期就树立了“为中华之崛起而读书”的远大理想，抱定了救国救比的信念，以极大的热忱投入到伟大的革命事业当中，最终成为新中国的缔造者之一。'], 'Answer': 'A', 'Q_id': '039501'}
{'Question': '在内容的选择与表达上，下面哪一首小诗与冰心的《繁星·春水》中的作品更为接近？', 'Choices': ['A. 母亲啊/您是繁茂的大树/我是归巢的鸟儿/暮色四合/我藏进您的心中', 'B. 母亲啊/您是繁茂的大树/树干上的条条斑痕/都是您辛劳岁月的见证'], 'Answer': 'B', 'Q_id': '077401'}
{'Question': '根据原文内容，下列说法不正确的一项是', 'Choices': ['A. 如果不对外开放，不实行“拿来主义” ，就没有中国改革开放四十年的快速发展。', 'B. 只要拿来主义者沉着，勇猛，有辨别，不自私，就一定能创造出新文艺'], 'Answer': 'B', 'Q_id': '156403'}


In [12]:
"""检查答案是否在选项中"""
for d in data:
    for q in d['Questions']:
        answer= q['Answer'].lower().strip()
        if not any([True if c.lower().strip().startswith(answer) else False for c in q['Choices']]):
            print(q)
print("processd")

processd


查看选项中是否有“以上全部”等字样；

找出选项中不含中文的

找出选项字符小于3的

In [13]:
import re
choice_space = re.compile("^[ABCDabcd]．?")

chinese = re.compile('[\u4e00-\u9fa5]+')

answer_no_chinese_number = 0
answer_no_chinese = []
for d in data:
    for q in d['Questions']:
        for c in q['Choices']:
            c = choice_space.sub('', c).strip()
            if not chinese.search(c):
                # print(q['Question'])
                # print("________")
                answer_no_chinese_number += 1
                answer_no_chinese.append(d)
                break
    
print(answer_no_chinese_number)

with open("check_text.json", 'w', encoding="utf-8") as f:
    f.write(json.dumps(answer_no_chinese,ensure_ascii=False))



223


解题思路 1

1. 对于一般选项的问题：如<br>
"下列对原文有关内容的分析和概括，不正确的一项是",<br>
"A、高祖之所以要缉拿季布， 是因为高祖对季布曾经使他身陷绝境耿耿于怀。",<br>
"B、朱家不仅冒险收留了季布，而且为他奔走，寻找理由，当季布最终被赦免后，他又悄然退避，显示出高尚的人格。",<br>
"C、丁公当年遇到身处危难之中的高祖，被高祖的好话所打动，不仅没有将高祖杀掉，还主动撤回了军队。",<br>
"D、对当年的仇敌，恨之入骨的季布最后被封官，有救命之恩的丁公却被找了个借口处死，高祖这样做，是从巩固自己的统治地位的角度考虑的。"<br>
对于这类题，采用[CLS] TEXT [SEP] Question Description [SEP] A 这种模型输入格式<br>
为了将其他选项纳入模型辅助判断该选项，选择输入为[CLS] B [SEP] C [SEP] D

2. 对于选项中带有序号的，如<br>
"下列句子，全都能表现愚公移山“意志坚定”的一组是①惩山北之塞，出入之也迂也 ②指通豫南，达于汉阴 ③寒暑易节，始一反焉 ④虽我之死，有子存焉 ⑤帝感其诚，命夸娥氏二子负二山",<br>
"A. ③④",<br>
"B. ②⑤",<br>
"C. ②③",<br>
"D. ①④"<br>
对于这类题采用[CLS] TEXT [SEP] Question Description [SEP] 惩山北之塞，出入之也迂也 [unused341] 虽我之死，有子存焉 <br>
为了将其他选项纳入模型辅助判断该选项，选择输入为[CLS] B [SEP] C [SEP] D, 其中B,C,D分别采用"指通豫南，达于汉阴 [unused341] 帝感其诚，命夸娥氏二子负二山" 这样的输入格式<br>

由于这些输入采用多个分段符号[SEP],目前预训练模型中支持多个分段符号的，只有XLNet模型<br>
将②③这些符号转化成bert词汇表中[unused12][unused13] 这些符号，ord("①")==9312,<br>
将选项中的空格用特殊符号进行替换[SPACE]<br>
将选项中的(1)这些编号用[unused51]替代，这是因为有这些编号中的context中，可能存在②这些字符。
不能混为一谈,要加以区分<br>
选项中的abcd这些选项，也使用从[unused51]开始替代。<br>
Q_id为274101和012602将删除<br>


In [58]:
import unicodedata
import re

def strQ2B(string):
    C_pun = u'，！？【】（）《》“”‘’；：．▏‖'
    E_pun = u',!?[]()<>""\'\';:.||'
    table= {ord(f):ord(t) for f,t in zip(C_pun, E_pun)}
    string = string.translate(table)

    # 转换说明：
    # 全角字符unicode编码从65281~65374 （十六进制 0xFF01 ~ 0xFF5E）
    # 半角字符unicode编码从33~126 （十六进制 0x21~ 0x7E）
    # 空格比较特殊，全角为 12288（0x3000），半角为 32（0x20）
    # 除空格外，全角/半角按unicode编码排序在顺序上是对应的（半角 + 0x7e= 全角）,所以可以直接通过用+-法来处理非空格数据，对空格单独处理。
    rstring = ""
    for uchar in string:
        # 返回赋予Unicode字符uchar的字符串型通用分类。
        inside_code = ord(uchar)

        if inside_code == 0 or inside_code == 0xfffd:
            continue
        cat = unicodedata.category(uchar)
        if cat == "Mn" or cat=='Cf' or cat=="So":
            continue
        if inside_code == 12288:  # 全角空格直接转换
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)
    return rstring

分析content的长度

In [54]:
spaces = re.compile("\s+")
contents_length = []
for d in data:
    content = d['Content']
    content = strQ2B(content)
    content = spaces.sub('', content)
    contents_length.append(len(content))

contents_length = sorted(contents_length)

print("最长的文本长度:",max(contents_length))
print("最短文本长度:",min(contents_length))
threshold = 1000

gt_threshold = list(filter(lambda x:x>threshold,contents_length))
print("大于{}的文本数量{}，\n占比{}".format(threshold, len(gt_threshold),len(gt_threshold)/len(contents_length)))
            

最长的文本长度: 2931
最短文本长度: 38
大于1000的文本数量3450，
占比0.5463182897862233


"ID": 126 文本中缺少标识符
"ID": 2741 选项中缺少标识


"ID": 2585 程序识别错误
"ID": 2978 中选项的数字是说明段数
"ID": 3947 程序处理有误
"ID": 5213
"ID": 4623
"ID": 4416


In [123]:
import unicodedata
import re
from collections import OrderedDict

compiles = re.compile("\([0-9A-Za-z]\)")
special_token_start_index = 9312 #①
special_token_start_word = "[unused%s]"
other_special_start_word = "[unused%s]"

chinese = re.compile('[\u4e00-\u9fa5,。:;]+')

def judge_choice_chinses(choices):
    """判断选项中，是否都没有中文"""
    had_chinses = False
    for c in choices:
        if chinese.search(c):
            had_chinses = True
        elif len("".join(number_compile.findall(c)))==len(c):
            had_chinses = True
    return had_chinses

alphabet_comiles = re.compile("[A-Za-z]+")
dot_number_compile = re.compile("[1-9]\.")
bracket_compile = re.compile("\([a-zA-Z0-9]\)")
def judge_choice_alphabet(choices,question,context):
    """检查选项中，是否含有字母，如果有，再次检测是否是拼英"""
    is_alpha = False
    is_from_question = False
    all_token_index = []
    all_token_from_context = []
    all_token_from_question = []
    if all([True if alphabet_comiles.search(c) else False for c in choices]) and all([True if c.find("(")==-1 else False for c in choices]):
        """在每个选项中都找到字母"""
        all_alphabet = sum([alphabet_comiles.findall(c) for c in choices],[])
        all_alphabet = set(sum([list(w) for w in all_alphabet],[]))
        all_specal_token = all_alphabet
        all_alphabet_index_question = [(question.find(w),w,len(w)) if question.find(w)>-1 else (-1,w,len(w)) for w in all_alphabet]
        all_alphabet_index_question = sorted(all_alphabet_index_question,key=lambda x:x[0])
        filters= list(filter(lambda x:x[0]!=-1,all_alphabet_index_question))
        for i,j in zip(filters[:-1],filters[1:]):
            if j[0]-i[0] < 2:
                is_alpha =True
        all_alphabet_index_context = [(context.find(w),w,len(w)) if context.find(w)>-1 else (-1,w,len(w)) for w in all_alphabet]
        all_alphabet_index_context = sorted(all_alphabet_index_context,key=lambda x:x[0])
        filters= list(filter(lambda x:x[0]!=-1,all_alphabet_index_context))
        for i,j in zip(filters[:-1],filters[1:]):
            if j[0]-i[0] < 2: #两个字母在文本很相近，说明其可能是拼英
                is_alpha =True
        if not is_alpha:
            all_token_from_context = all_alphabet_index_context
            all_token_from_question = all_alphabet_index_question
            
    if is_alpha or (len(all_token_from_context) == 0 and len(all_token_from_question)==0):
        if any([True if 9312<=ord(w)<9362 else False for c in choices for w in c]):
            """每个选项中③④⑤"""
            all_specal_token = set([a for c in choices for a in c if 9312<=ord(a)<9362])
            all_special_from_question = [(question.find(w),w,len(w)) if question.find(w)>-1 else (-1,w,len(w)) for w in all_specal_token]
            all_special_fom_context = [(context.find(w),w,len(w)) if context.find(w)>-1 else (-1,w,len(w)) for w in all_specal_token]
            all_token_from_question = all_special_from_question
            all_token_from_context = all_special_fom_context
        elif any([True if dot_number_compile.search(c) else False for c in choices]):
            number_compile = re.compile("[1-9]\.?")
            all_specal_token = sum([number_compile.findall(c) for c in choices],[])
            all_number_from_question = [(question.find(w),w,len(w)) if question.find(w)>-1 else (-1,w,len(w)) for w in all_specal_token]
            all_number_fom_context = [(context.find(w),w,len(w)) if context.find(w)>-1 else (-1,w,len(w)) for w in all_specal_token]
            all_token_from_question = all_number_from_question
            all_token_from_context = all_number_fom_context
        elif any([True if bracket_compile.search(c) else False for c in choices]):
            all_specal_token = sum([bracket_compile.findall(c) for c in choices],[])
            all_bracket_from_question = [(question.find(w),w,len(w)) if question.find(w)>-1 else (-1,w,len(w)) for w in all_specal_token]
            all_bracket_fom_context = [(context.find(w),w,len(w)) if context.find(w)>-1 else (-1,w,len(w)) for w in all_specal_token]
            all_token_from_question = all_bracket_from_question
            all_token_from_context = all_bracket_fom_context
        else:
            raise("")
    all_token_from_question = list(filter(lambda x:x[0]!=-1,all_token_from_question))
    all_token_from_context = list(filter(lambda x:x[0]!=-1,all_token_from_context))
    if len(all_token_from_question)==len(all_specal_token):
        all_token_index = all_token_from_question
        is_from_question = True
    elif len(all_token_from_context) == len(all_specal_token):
        all_token_index = all_token_from_context
    else:
        print(all_token_from_context)
        print(all_token_from_question)
        raise ("ValueError")
            

    return is_alpha,all_token_index,is_from_question

    
            
choice_space = re.compile("^[ABCDabcd]\s?(\.|、|。)?")


alpha_compile = re.compile("[a-zA-Z0-9]")
alphabet = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
number_compile = re.compile('[0-9]+')
def sub_bracket_number(w):
    """将选项中(1)(2) 或者abcd这些编号替换成[unused5？]从51开始的编号"""
    if len(w)!=1:
        w = w.replace(")","").replace("(","").replace(".","")
    w = w.lower()
    if number_compile.search(w):
        num = number_compile.findall(w)[0]
        other_special_word = other_special_start_word%str(50+int(num))
        return other_special_word
    elif w.lower() in 'abcdefghijklmnopqrstuvwxyz':
        num = alphabet[w]
        other_special_word = other_special_start_word%str(50+num)
        return other_special_word
    elif special_token_start_index <= ord(w) <=9362:
        """①④⑥ 这些替换成[unused1？]，从11开始的特殊字符"""
        special_token_index = ord(w)-special_token_start_index+1
        special_token_word = special_token_start_word%str(10+special_token_index)
        return special_token_word
    else:
        print(w)
        raise ValueError

with open("validation.json", 'r', encoding="utf-8") as f:
    answer_no_chinese = json.load(f)


new_data = []
will_drop_q_id = ["012602","274101"]
# 训练集中，297802是询问自然段

for d in answer_no_chinese:
    new_d = OrderedDict()
    text = d['Content']
    text = strQ2B(text)
    ID = d['ID']
    all_token_from_contexts = []
    Questions= []
    for q in d['Questions']:
        new_q = OrderedDict()
        question = q['Question']
        question = strQ2B(question)
        choices = q['Choices']
        # answer_id = q['Answer']
        Q_id = q['Q_id']
        for index, c in enumerate(choices):
            choices[index] =re.sub("\s+","",choice_space.sub('',strQ2B(c)).strip())
        if not judge_choice_chinses(choices) and q['Q_id']  not in will_drop_q_id:
            new_choices = []
            is_alpha,all_token_from_question,is_from_question = judge_choice_alphabet(choices,question,text)
            if len(all_token_from_question):
                if is_from_question:
                    # print(all_token_from_question)
                    all_token_from_question = sorted(all_token_from_question,key=lambda x:x[0],reverse=True)
                    all_alphabet = [w for _,w,_ in all_token_from_question]
                    for i,w,length in all_token_from_question:
                        su_word = sub_bracket_number(w)
                        question = question[:int(i)]+su_word+question[int(i)+int(length):]
                else:
                    all_token_from_context = sorted(all_token_from_question,key=lambda x:x[0],reverse=True)
                    all_alphabet = [w for _,w,_ in all_token_from_context]
                    all_token_from_contexts.extend(all_token_from_context)
                    # print(all_token_from_context)

                for c in choices:
                    ab={}
                    for a in all_alphabet:
                        if c.find(a)>-1:
                            c_index = c.find(a)
                            ab[a]=c_index
                    for w,i in sorted(ab.items(),key=lambda x:x[1],reverse=True):
                        c= c[:i] + sub_bracket_number(w) +c[i+len(w):]
                    
                    new_choices.append(c)
            else:
                print(d['ID'])

                print(choices)
            choices = new_choices

        new_q['Questions']= question.lower()
        new_q['Choices'] = choices
        # new_q['Answer'] = answer_id
        new_q['Q_id'] = Q_id
        Questions.append(new_q)
        
    if len(all_token_from_contexts):
        all_token_from_contexts = sorted(set(all_token_from_contexts),key=lambda x:x[0],reverse=True)
        
        for i,w,length in all_token_from_contexts:
            su_word = sub_bracket_number(w)
            if w != text[i:i+length]:
                # print(d['ID'],text[i:i+length],w)
                # print(all_token_from_contexts)
                break
            text = text[:int(i)]+su_word+text[int(i)+int(length):]

    new_d["ID"] = ID
    new_d['Content'] = text
    new_d['Questions'] = Questions
    new_data.append(new_d)

with open("validation_v2.json", 'w', encoding="utf-8") as f:
    f.write(json.dumps(new_data,ensure_ascii=False))

['(1)', '(2)']